In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/semantic_parsing/tree_slu/main')

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 52kB/s 
     |████████████████████████████████| 61kB 27.1MB/s 
     |████████████████████████████████| 419kB 50.7MB/s 
     |████████████████████████████████| 3.0MB 38.6MB/s 


In [0]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import nltk
import os
import random

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (source, target_in, target_out)

In [0]:
def dataset(is_training, params):
  _shapes = ([None], [None], [None])
  _types = (tf.int32, tf.int32, tf.int32)
  _pads = (0, 0, 0)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['train_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['eval_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.embedding = tf.Variable(np.load('../vocab/word.npy'),
                                 dtype=tf.float32,
                                 name='pretrained_embedding')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.encoder = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(params['rnn_units']//2,
                                                                     return_state=True,
                                                                     return_sequences=True))
  
  
  def call(self, inputs, training=False):
    batch_sz = tf.shape(inputs)[0]
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    
    self.dropout.input_noise_shape = (batch_sz, 1, 300)
    x = self.dropout(x, training=training)
    
    encoder_outputs, state_fw, state_bw = self.encoder(x)
    return (encoder_outputs, [tf.concat((state_fw, state_bw), -1)])

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units, use_bias=False)
    self.W2 = tf.keras.layers.Dense(units, use_bias=False)
    self.V = tf.keras.layers.Dense(1, use_bias=False)

  def call(self, query, values, values_mask):
    query = tf.expand_dims(query, 1)

    score = self.V(tf.tanh(self.W1(values) + self.W2(query)))
    score = tf.squeeze(score, -1)

    # pre-softmax masking
    paddings = tf.fill(tf.shape(score), float('-inf'))
    score = tf.where(tf.equal(values_mask, 0), paddings, score)
    
    align = tf.nn.softmax(score, axis=1)
    align = tf.expand_dims(align, -1)
    
    context_vector = tf.matmul(values, align, transpose_a=True)
    context_vector = tf.squeeze(context_vector, -1)

    return context_vector

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, params, tied_embedding):
    super().__init__()
    self.embedding = tied_embedding
    self.attention = BahdanauAttention(params['rnn_units'])
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.cell = tf.keras.layers.StackedRNNCells([
      tf.keras.layers.GRUCell(params['rnn_units']),
      tf.keras.layers.GRUCell(params['rnn_units']),
    ])
    self.out_bias = self.add_weight(name='out_bias', shape=[len(params['tgt2idx'])+1])
  
  
  def call(self, inputs, training=False):
    inputs, states, memory, memory_mask = inputs
    
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    
    h0, h1 = tf.split(states, 2, axis=-1)
    
    context_vector = self.attention(h1, memory, memory_mask)
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    
    x = tf.concat([context_vector, x], axis=-1)
    
    x = self.dropout(x, training=training)
    
    output, states = self.cell(x, (h0, h1))
    
    logits = tf.matmul(output, self.embedding, transpose_b=True)
    logits = tf.nn.bias_add(logits, self.out_bias)
    
    states = tf.concat([states[0], states[1],], axis=-1)
    
    return logits, states

In [0]:
def train_step(source, target_in, target_out, encoder, decoder, params):
  loss = 0
  logits = []
  encoder_outputs, decoder_state = encoder(source, training=True)
  decoder_state = tf.concat(decoder_state + decoder_state, -1)
  
  for t in range(target_in.shape[1]):
    _logits, decoder_state = decoder([target_in[:, t],
                                      decoder_state,
                                      encoder_outputs,
                                      tf.sign(source)],
                                      training=True)
    logits.append(_logits)
  
  logits = tf.stack(logits, 1)
  
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=target_out, logits=logits)
  weights = tf.cast(tf.sign(target_in), tf.float32)
  loss = tf.reduce_sum(loss * weights) / tf.reduce_sum(weights)
  
  return loss

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True

In [0]:
params = {
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model/',
    'dropout_rate': 0.2,
    'rnn_units': 300,
    'max_decode_len': 50,
    'lr': 4e-4,
    'clip_norm': 10.,
    'buffer_size': 31279,
    'train_batch_size': 32,
    'eval_batch_size': 128,
    'num_patience': 5,
}

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [0]:
Path(params['model_path']).mkdir(exist_ok=True)

encoder = Encoder(params)
encoder.build((None, None))
pprint.pprint([(v.name, v.shape) for v in encoder.trainable_variables])

decoder = Decoder(params, encoder.embedding)
decoder.build([[None], [None, 2*params['rnn_units']], [None, None, params['rnn_units']], [None, None]])
pprint.pprint([(v.name, v.shape) for v in decoder.trainable_variables])

[('bidirectional/forward_unified_gru/kernel:0', TensorShape([300, 450])),
 ('bidirectional/forward_unified_gru/recurrent_kernel:0',
  TensorShape([150, 450])),
 ('bidirectional/forward_unified_gru/bias:0', TensorShape([2, 450])),
 ('bidirectional/backward_unified_gru/kernel:0', TensorShape([300, 450])),
 ('bidirectional/backward_unified_gru/recurrent_kernel:0',
  TensorShape([150, 450])),
 ('bidirectional/backward_unified_gru/bias:0', TensorShape([2, 450])),
 ('pretrained_embedding:0', TensorShape([8692, 300]))]
[('bahdanau_attention/dense/kernel:0', TensorShape([300, 300])),
 ('bahdanau_attention/dense_1/kernel:0', TensorShape([300, 300])),
 ('bahdanau_attention/dense_2/kernel:0', TensorShape([300, 1])),
 ('stacked_rnn_cells/kernel:0', TensorShape([600, 900])),
 ('stacked_rnn_cells/recurrent_kernel:0', TensorShape([300, 900])),
 ('stacked_rnn_cells/bias:0', TensorShape([900])),
 ('stacked_rnn_cells/kernel:0', TensorShape([300, 900])),
 ('stacked_rnn_cells/recurrent_kernel:0', TensorSh

In [0]:
decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.96)
optim = tf.optimizers.Adam(params['lr'])
global_step = 14670
encoder.load_weights(os.path.join(params['model_path'], 'encoder_{}'.format(global_step)))
decoder.load_weights(os.path.join(params['model_path'], 'decoder_{}'.format(global_step)))

In [0]:
history_acc = []
best_acc = .0

In [0]:
t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

In [0]:
def minimal_test(encoder, decoder, params):
  test_str = ['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']
  test_arr = tf.convert_to_tensor([[params['tgt2idx'][w] for w in test_str]])
  generated = tf.convert_to_tensor([[1]])
  ids = generated[0]
  
  encoder_outputs, decoder_state = encoder(test_arr, training=False)
  decoder_state = tf.concat(decoder_state + decoder_state, -1)
  
  for i in range(params['max_decode_len']):
    logits, decoder_state = decoder([ids,
                                     decoder_state,
                                     encoder_outputs,
                                     tf.sign(test_arr)],
                                     training=False)
    ids = tf.argmax(logits, axis=-1, output_type=tf.int32)
    generated = tf.concat((generated, tf.expand_dims(ids, 1)), axis=1)
    if np.asscalar(ids.numpy()) == 2:
      break
  
  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str))
  parsed = ' '.join([params['idx2tgt'][idx] for idx in generated[0].numpy()[1:-1]])
  print('parsed:', parsed)
  print()
  try:
    nltk.tree.Tree.fromstring(parsed.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
variables = encoder.trainable_variables + decoder.trainable_variables

while True:
  # TRAINING
  for (source, target_in, target_out) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      loss = train_step(source, target_in, target_out, encoder, decoder, params)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    
    global_step += 1
  
  # EVALUATION
  is_training=False
  minimal_test(encoder, decoder, params)
  m = tf.keras.metrics.Mean()
  
  for i, (source, target_in, target_out) in enumerate(dataset(is_training=is_training, params=params)):
    generated = tf.ones((source.shape[0], 1), tf.int32)
    ids = tf.squeeze(generated, axis=1)
    encoder_outputs, decoder_state = encoder(source, training=is_training)
    decoder_state = tf.concat(decoder_state + decoder_state, -1)
    
    for j in range(target_out.shape[1]):
      logits, decoder_state = decoder([ids,
                                       decoder_state,
                                       encoder_outputs,
                                       tf.sign(source)],
                                       training=is_training)
      ids = tf.argmax(logits, axis=1, output_type=tf.int32)
      generated = tf.concat((generated, tf.expand_dims(ids, 1)), axis=1)

    seq_lens = tf.argmax(tf.cast(tf.equal(target_out, 2), tf.int32), axis=1)
    for pred, tgt, seq_len in zip(generated.numpy(), target_out.numpy(), seq_lens.numpy()):
      pred = pred[1:][:seq_len+1]
      tgt = tgt[:seq_len+1]
      matched = np.all(pred == tgt)
      m.update_state(int(matched))
  
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Exact Match Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
    encoder.save_weights(os.path.join(params['model_path'], 'encoder_{}'.format(global_step)))
    decoder.save_weights(os.path.join(params['model_path'], 'decoder_{}'.format(global_step)))
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0516 00:33:32.826574 140231268951936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Reading ../data/train.tsv


I0516 00:34:35.006713 140231268951936 interactiveshell.py:2882] Step 14700 | Loss: 0.0268 | Spent: 62.3 secs | LR: 0.000220
I0516 00:36:05.501112 140231268951936 interactiveshell.py:2882] Step 14750 | Loss: 0.0231 | Spent: 90.5 secs | LR: 0.000219
I0516 00:37:32.498473 140231268951936 interactiveshell.py:2882] Step 14800 | Loss: 0.0144 | Spent: 87.0 secs | LR: 0.000219
I0516 00:39:00.578161 140231268951936 interactiveshell.py:2882] Step 14850 | Loss: 0.0159 | Spent: 88.1 secs | LR: 0.000218
I0516 00:40:35.499025 140231268951936 interactiveshell.py:2882] Step 14900 | Loss: 0.0124 | Spent: 94.9 secs | LR: 0.000218
I0516 00:42:01.602390 140231268951936 interactiveshell.py:2882] Step 14950 | Loss: 0.0145 | Spent: 86.1 secs | LR: 0.000217
I0516 00:43:29.713579 140231268951936 interactiveshell.py:2882] Step 15000 | Loss: 0.0231 | Spent: 88.1 secs | LR: 0.000217
I0516 00:45:00.504919 140231268951936 interactiveshell.py:2882] Step 15050 | Loss: 0.0151 | Spent: 90.8 secs | LR: 0.000216
I0516 00

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are [ sl:category_event the nutcracker show ] playing [ sl:location [ in:get_location [ sl:search_radius near ] [ sl:location_user me ] ] ] ]

                       in:get_event                                                                        
  __________________________|_____________________________________________________                          
 |     |    |     |                        |                                 sl:location                   
 |     |    |     |                        |                                      |                         
 |     |    |     |                        |                               in:get_location                 
 |     |    |     |                        |                      ________________|_______________          
 |     |    |     |                 sl:category_even      sl:search_radius          

I0516 01:04:08.637423 140231268951936 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.712
I0516 01:04:08.804716 140231268951936 interactiveshell.py:2882] Best Accuracy: 0.712


...

I0516 09:07:36.672530 140231268951936 interactiveshell.py:2882] Step 31300 | Loss: 0.0105 | Spent: 158.2 secs | LR: 0.000111
I0516 09:09:03.214328 140231268951936 interactiveshell.py:2882] Step 31350 | Loss: 0.0027 | Spent: 86.5 secs | LR: 0.000111
I0516 09:10:25.617316 140231268951936 interactiveshell.py:2882] Step 31400 | Loss: 0.0085 | Spent: 82.4 secs | LR: 0.000111
I0516 09:11:56.191527 140231268951936 interactiveshell.py:2882] Step 31450 | Loss: 0.0023 | Spent: 90.6 secs | LR: 0.000111
I0516 09:13:28.524062 140231268951936 interactiveshell.py:2882] Step 31500 | Loss: 0.0098 | Spent: 92.3 secs | LR: 0.000111
I0516 09:14:59.846086 140231268951936 interactiveshell.py:2882] Step 31550 | Loss: 0.0041 | Spent: 91.3 secs | LR: 0.000110
I0516 09:16:28.502799 140231268951936 interactiveshell.py:2882] Step 31600 | Loss: 0.0030 | Spent: 88.7 secs | LR: 0.000110
I0516 09:17:57.536278 140231268951936 interactiveshell.py:2882] Step 31650 | Loss: 0.0024 | Spent: 89.0 secs | LR: 0.000110
I0516 0

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are [ sl:category_event the nutcracker show ] playing [ sl:location [ in:get_location [ sl:search_radius near ] [ sl:location_user me ] ] ] ]

                       in:get_event                                                                        
  __________________________|_____________________________________________________                          
 |     |    |     |                        |                                 sl:location                   
 |     |    |     |                        |                                      |                         
 |     |    |     |                        |                               in:get_location                 
 |     |    |     |                        |                      ________________|_______________          
 |     |    |     |                 sl:category_even      sl:search_radius          

I0516 09:37:26.179867 140231268951936 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.726
I0516 09:37:26.323935 140231268951936 interactiveshell.py:2882] Best Accuracy: 0.726


Reading ../data/train.tsv


KeyboardInterrupt: ignored